In [1]:
# Import required libraries.

import pandas as pd

In [2]:
# Read all datasets.

jan  = pd.read_csv("datasets/01_flight_details_january.csv")
feb  = pd.read_csv("datasets/02_flight_details_february.csv")
mar  = pd.read_csv("datasets/03_flight_details_march.csv")
apr  = pd.read_csv("datasets/04_flight_details_april.csv")
may  = pd.read_csv("datasets/05_flight_details_may.csv")
jun  = pd.read_csv("datasets/06_flight_details_june.csv")
jul  = pd.read_csv("datasets/07_flight_details_july.csv")
aug  = pd.read_csv("datasets/08_flight_details_august.csv")
sep  = pd.read_csv("datasets/09_flight_details_september.csv")
octo = pd.read_csv("datasets/10_flight_details_october.csv")
nov  = pd.read_csv("datasets/11_flight_details_november.csv")
dec  = pd.read_csv("datasets/12_flight_details_december.csv")


In [12]:
# Make list of dasets to make iterating easy.

df_list = [jan, feb, mar, may, jun, jul, aug, sep, octo, nov, dec]

In [13]:
feb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410517 entries, 0 to 410516
Data columns (total 25 columns):
FL_DATE                410517 non-null object
AIRLINE_ID             410517 non-null int64
TAIL_NUM               409907 non-null object
FL_NUM                 410517 non-null int64
ORIGIN                 410517 non-null object
ORIGIN_CITY_NAME       410517 non-null object
DEST                   410517 non-null object
DEST_CITY_NAME         410517 non-null object
CRS_DEP_TIME           410517 non-null int64
DEP_TIME               404337 non-null float64
DEP_DELAY              404335 non-null float64
DEP_DELAY_GROUP        404335 non-null float64
CRS_ARR_TIME           410517 non-null int64
ARR_TIME               403966 non-null float64
ARR_DELAY              403225 non-null float64
ARR_DELAY_GROUP        403225 non-null float64
CANCELLED              410517 non-null float64
DIVERTED               410517 non-null float64
AIR_TIME               403225 non-null float64
CARRIER_DE

In [75]:
# Make copies of datasets.
df_copy_list = []
for df in df_list:
    df_copy_list.append(df.copy())

# Deconstruct copy list for redable access to elements.
jan_copy, feb_copy, mar_copy, may_copy, jun_copy, jul_copy, aug_copy, sep_copy, octo_copy, nov_copy, dec_copy = df_copy_list

In [76]:
# Convert columns to appropriate datatypes.

for df in df_copy_list:
    
    df.dropna(subset=["DEP_TIME", "ARR_TIME"], inplace=True)
    
    df.FL_DATE = pd.to_datetime(df.FL_DATE)
    
    df.AIRLINE_ID = df.AIRLINE_ID.astype("str")
    
    df.FL_NUM = df.FL_NUM.astype("str")
    
    df.CRS_DEP_TIME = df.CRS_DEP_TIME.astype("str").str.zfill(4).replace("2400", "0000")
    df.CRS_ARR_TIME = df.CRS_ARR_TIME.astype("str").str.zfill(4).replace("2400", "0000")    
    df.CRS_DEP_TIME = pd.to_datetime(df.CRS_DEP_TIME, format="%H%M").dt.time
    df.CRS_ARR_TIME = pd.to_datetime(df.CRS_ARR_TIME, format="%H%M").dt.time
    
    df.DEP_TIME = df.DEP_TIME.astype("int").astype("str").str.zfill(4).replace("2400", "0000")
    df.ARR_TIME = df.ARR_TIME.astype("int").astype("str").str.zfill(4).replace("2400", "0000")    
    df.DEP_TIME = pd.to_datetime(df.DEP_TIME, format="%H%M").dt.time
    df.ARR_TIME = pd.to_datetime(df.ARR_TIME, format="%H%M").dt.time
    
    df.DEP_DELAY_GROUP = df.DEP_DELAY_GROUP.astype("int", errors="ignore")
    df.ARR_DELAY_GROUP = df.ARR_DELAY_GROUP.astype("int", errors="ignore")
    
    df.DEP_DELAY = df.DEP_DELAY.astype("int", errors="ignore")
    df.ARR_DELAY = df.ARR_DELAY.astype("int", errors="ignore")
    
    df.CANCELLED = df.CANCELLED.astype("bool")
    df.DIVERTED = df.DIVERTED.astype("bool")
    
    df.drop("Unnamed: 24", axis=1, inplace=True)

In [86]:
feb_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403966 entries, 0 to 410516
Data columns (total 24 columns):
FL_DATE                403966 non-null datetime64[ns]
AIRLINE_ID             403966 non-null object
TAIL_NUM               403966 non-null object
FL_NUM                 403966 non-null object
ORIGIN                 403966 non-null object
ORIGIN_CITY_NAME       403966 non-null object
DEST                   403966 non-null object
DEST_CITY_NAME         403966 non-null object
CRS_DEP_TIME           403966 non-null object
DEP_TIME               403966 non-null object
DEP_DELAY              403966 non-null int32
DEP_DELAY_GROUP        403966 non-null int32
CRS_ARR_TIME           403966 non-null object
ARR_TIME               403966 non-null object
ARR_DELAY              403225 non-null float64
ARR_DELAY_GROUP        403225 non-null float64
CANCELLED              403966 non-null bool
DIVERTED               403966 non-null bool
AIR_TIME               403225 non-null float64
CARRIER_DE

In [88]:
feb_copy.head()

,FL_DATE,AIRLINE_ID,TAIL_NUM,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,...,ARR_DELAY,ARR_DELAY_GROUP,CANCELLED,DIVERTED,AIR_TIME,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2017-02-01,20409,N267JB,33,BTV,"Burlington, VT",JFK,"New York, NY",19:07:00,18:57:00,...,-8.0,-1.0,False,False,66.0,NaN,NaN,NaN,NaN,NaN
1,2017-02-01,20409,N267JB,34,JFK,"New York, NY",BTV,"Burlington, VT",17:06:00,16:56:00,...,-13.0,-1.0,False,False,49.0,NaN,NaN,NaN,NaN,NaN
2,2017-02-01,20409,N632JB,42,LGB,"Long Beach, CA",RNO,"Reno, NV",09:12:00,09:09:00,...,-9.0,-1.0,False,False,63.0,NaN,NaN,NaN,NaN,NaN
3,2017-02-01,20409,N632JB,43,RNO,"Reno, NV",LGB,"Long Beach, CA",11:20:00,11:19:00,...,-8.0,-1.0,False,False,64.0,NaN,NaN,NaN,NaN,NaN
4,2017-02-01,20409,N351JB,49,BOS,"Boston, MA",SAV,"Savannah, GA",10:40:00,10:33:00,...,-12.0,-1.0,False,False,141.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
for df in df_copy_list:
    df["ORIGIN_CITY"], df["ORIGIN_STATE"] = df.ORIGIN_CITY_NAME.str.split(", ").str
    df["DEST_CITY"], df["DEST_STATE"] = df.DEST_CITY_NAME.str.split(", ").str
    df.drop(["ORIGIN_CITY_NAME", "DEST_CITY_NAME"], axis=1)

In [107]:
df.head()

,FL_DATE,AIRLINE_ID,TAIL_NUM,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,...,CANCELLED,DIVERTED,AIR_TIME,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,ORIGIN_CITY,ORIGIN_STATE
0,2017-12-01,21171,N854VA,1001,SFO,"San Francisco, CA",DCA,"Washington, DC",08:00:00,08:52:00,...,False,False,266.0,45.0,0.0,0.0,0.0,0.0,San Francisco,CA
1,2017-12-01,21171,N854VA,1002,DCA,"Washington, DC",SFO,"San Francisco, CA",17:10:00,17:28:00,...,False,False,341.0,NaN,NaN,NaN,NaN,NaN,Washington,DC
2,2017-12-01,21171,N842VA,1011,JFK,"New York, NY",SFO,"San Francisco, CA",07:30:00,07:21:00,...,False,False,368.0,NaN,NaN,NaN,NaN,NaN,New York,NY
3,2017-12-01,21171,N845VA,1012,SFO,"San Francisco, CA",JFK,"New York, NY",07:05:00,06:53:00,...,False,False,285.0,NaN,NaN,NaN,NaN,NaN,San Francisco,CA
4,2017-12-01,21171,N851VA,1022,SFO,"San Francisco, CA",JFK,"New York, NY",09:25:00,10:35:00,...,False,False,279.0,0.0,0.0,49.0,0.0,0.0,San Francisco,CA
